# Used code to upload data from database file after processing by ELMOD

In [13]:
import pandas as pd
import pyodbc
import os
import re
from tkinter import simpledialog

batch_no = simpledialog.askinteger (title= "Batch No", prompt= "Please enter the batch no:")

# Location of the access file
mdb_file = r"C:\Users\Khaled Ahmed\Desktop\fwd_analysis_ASH\003_Databases\anaylsis_ash_08-2024_batch_1.mde"
        
# Create a connection string
conn_string = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=' + mdb_file + ';'
        )
        
# The tables
table_name_list = ['Deflections', 'GPS', 'MODULI_ESTIMATED', 'Thickness', 'Materials']
df_access_dict = {}
        
# Create connection
with pyodbc.connect(conn_string) as connection:
    cursor = connection.cursor()
    try:
        for table in table_name_list:
            query = f'SELECT * FROM {table}'
            df = pd.read_sql(query, connection)
            df['batch_no'] = batch_no
            df_access_dict[table] = df

    finally:
        cursor.close()




C:\Users\Khaled Ahmed\AppData\Local\Temp\ipykernel_32200\1817268556.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
C:\Users\Khaled Ahmed\AppData\Local\Temp\ipykernel_32200\1817268556.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
C:\Users\Khaled Ahmed\AppData\Local\Temp\ipykernel_32200\1817268556.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
C:\Users\Khaled Ahmed\AppData\Local\Temp\ipykernel_32200\1

# Used code to upload data to database

In [15]:
#To upload the data to the database 
from sqlalchemy import create_engine
user = 'postgres'
password = 'mjesbb51187'
database = 'elmod_db'
host = 'localhost'
port = '5432'

engine = create_engine (f'postgresql://{user}:{password}@{host}:{port}/{database}')

df_dict = {
    'deflections': df_access_dict['Deflections'],
    'gps': df_access_dict['GPS'],
    'moduli':df_access_dict['MODULI_ESTIMATED'],
    'thickness':df_access_dict['Thickness'],
    'materials':df_access_dict['Materials'],
} 

with engine.connect() as connection:
    for table_name,df_db in df_dict.items():
        df_db.to_sql (table_name, connection , if_exists = 'append', index = False)